# VGG

In [74]:
import tensorflow as tf

# 1 模型构建

## （1）VGG块的构建

In [75]:
def vgg_block(num_conv, num_filters):
    # 序列模型
    blk = tf.keras.models.Sequential()

    # 遍历卷积层
    for _ in range(num_conv):
        blk.add(tf.keras.layers.Conv2D(num_filters, kernel_size = 3, padding = 'same', activation = 'relu'))

    # 最后一层是全连接层
    blk.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

    return blk

## (2)构建模型

In [76]:
def vgg(conv_arch):
    # 序列模型
    net = tf.keras.models.Sequential()

    # 生成卷积部分
    for (num_conv, num_filters) in conv_arch:
        net.add(vgg_block(num_conv, num_filters))

    # 全连接层
    net.add(tf.keras.models.Sequential([
        # 展平
        tf.keras.layers.Flatten(),
        # 全连接层
        tf.keras.layers.Dense(4096, activation = 'relu'),
        # 随机失活
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(4096, activation = 'relu'),

        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(10, activation = 'softmax')
    ]))

    return net

In [77]:
# 卷积块的参数
conv_arch = ((2, 64), (2, 128), (3, 256), (3, 512), (3, 512))

In [78]:
# 网络实例化
net = vgg(conv_arch)

In [79]:
X = tf.random.uniform((1, 224, 224, 1))
y = net(X)

net.summary()

Model: "sequential_52"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_53 (Sequential)      │ (1, 112, 112, 64)      │        37,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_54 (Sequential)      │ (1, 56, 56, 128)       │       221,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_55 (Sequential)      │ (1, 28, 28, 256)       │     1,475,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_56 (Sequential)      │ (1, 14, 14, 512)       │     5,899,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_57 (Sequential)      │ (1, 7, 7, 512)         │     7,079,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_58 (Sequential)      │ (1, 10)                │   119,586,826 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,300,362 (512.32 MB)

 Trainable params: 134,300,362 (512.32 MB)

 Non-trainable params: 0 (0.00 B)

# 2 数据读取

In [80]:
import numpy as np
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# N H W C
train_images = np.reshape(train_images, (train_images.shape[0], train_images.shape[1], train_images.shape[2], 1))

test_images = np.reshape(test_images, (test_images.shape[0], test_images.shape[1], test_images.shape[2], 1))

In [81]:
# 定义两个方法随机抽取部分样本演示

def get_train(size):
    index = np.random.randint(0, np.shape(train_images)[0], size)

    resize_images = tf.image.resize_with_pad(train_images[index], 224, 224, )

    return resize_images.numpy(), train_labels[index]

def get_test(size):
    index = np.random.randint(0, np.shape(test_images)[0], size)

    resize_images = tf.image.resize_with_pad(test_images[index], 224, 224, )

    return resize_images.numpy(), test_labels[index]

In [82]:
# 获取训练样本和测试样本
train_image, train_label = get_train(256)
test_image, test_label = get_test(128)

# 3 模型编译

In [83]:
# 优化器，损失函数，评价指标
net.compile(optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01),
            loss = tf.keras.losses.sparse_categorical_crossentropy,
            metrics = ['accuracy'])

# 4 模型训练

In [84]:
net.fit(train_images, train_labels, batch_size = 128, epochs = 3, verbose = 1, validation_split = 0.1)

Epoch 1/3


ValueError: Exception encountered when calling MaxPooling2D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node sequential_52_1/sequential_57_1/max_pooling2d_44_1/MaxPool2d}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](sequential_52_1/sequential_57_1/conv2d_118_1/Relu)' with input shapes: [?,1,1,512].[0m

Arguments received by MaxPooling2D.call():
  • inputs=tf.Tensor(shape=(None, 1, 1, 512), dtype=float32)

# 5 模型评估

In [85]:
net.evaluate(test_images, test_labels, verbose = 1)

ValueError: Exception encountered when calling MaxPooling2D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node sequential_52_1/sequential_57_1/max_pooling2d_44_1/MaxPool2d}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](sequential_52_1/sequential_57_1/conv2d_118_1/Relu)' with input shapes: [?,1,1,512].[0m

Arguments received by MaxPooling2D.call():
  • inputs=tf.Tensor(shape=(None, 1, 1, 512), dtype=float32)